## In-Class Assignment
<br/>
#### Jaime Guevara
#### March 27, 2019
#### CS-497

In [2]:
train_dir = "./fruits_data_set/Training/"
validation_dir = "./fruits_data_set/Validation/"

In [3]:
from keras import layers
from keras import models

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(25, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 128)         147584    
__________

In [13]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc'])

In [14]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(100, 100),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(100, 100),
        batch_size=20,
        class_mode='categorical')

Found 11585 images belonging to 25 classes.
Found 3867 images belonging to 25 classes.


In [15]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 100, 100, 3)
labels batch shape: (20, 25)


In [17]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=500,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/2
500/500 [==============================] - 155s 311ms/step - loss: 0.0782 - acc: 0.9796 - val_loss: 0.1652 - val_acc: 0.9590
Epoch 2/2
500/500 [==============================] - 153s 306ms/step - loss: 0.0570 - acc: 0.9860 - val_loss: 0.2474 - val_acc: 0.9480


In [20]:
from keras.applications.vgg16 import VGG16

In [22]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(100,100,3))


58892288/58889256 [==============================] - 5s 0us/step


<bound method Network.summary of <keras.engine.training.Model object at 0x109acc550>>

In [23]:
conv_base.trainable = False
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [24]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(25, activation='softmax'))

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               2359808   
_________________________________________________________________
dense_6 (Dense)              (None, 25)                12825     
Total params: 17,087,321
Trainable params: 2,372,633
Non-trainable params: 14,714,688
_________________________________________________________________


In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc'])

In [27]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=5,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/1
5/5 [==============================] - 119s 24s/step - loss: 3.4981 - acc: 0.1900 - val_loss: 3.1649 - val_acc: 0.1915
